<table>
  <tr>
    <td><div align="left"><font size="30">Finding fiducial markers</font></div></td>
    <td><img src="https://github.com/Tobias-Fischer/RVSS2022/blob/main/Robotic_Vision/common/RVSS-logo.png?raw=1" width="400"></td>
  </tr>
</table>

## Configuring CoLab
To make this run nicely in CoLab we have to install a bunch of packages. This will be slow and at the end you will need to restart the kernel. Push the RESTART KERNEL button that appears or from the menu above Runtime/RestartRuntime

In [ ]:
!pip install git+https://github.com/petercorke/spatialmath-python
!pip install git+https://github.com/petercorke/machinevision-toolbox-python@wip
!pip install mvtb-data
# !pip install -U ipython

Now, we can import some of the packages we need throughout, Matplotlib and NumPy, and configure the CoLab environment a bit.

In [ ]:
%matplotlib inline

import numpy as np
import math
from spatialmath import SE3, SO3
from spatialmath.base import e2h, h2e
from machinevisiontoolbox import Image, CentralCamera
from spatialmath.base import plot_point
import matplotlib.pyplot as plt


Fiducial markers are an effective way to mark points in the environment.  The markers carry a numerical identity and their position, or even pose, with respect to the camera can be determined.

This example uses OpenCV function which are available via the Machine Vision Toolbox.  Two common types of fiducual markers are used in robotics and vision: ApriTags (not covered here) and arUco markers.

## ArUco markers

ArUco markers were developed as a simple way to mark things for augmented reality (AR) applications, hence the AR in their name.  They are just high-contrast grids that encode a number as well as position and orientation.

We will load a synthetic image of some arUco markers in the lab.

In [ ]:
scene = Image.Read("lab-scene.png", rgb=False)
scene.disp();
print(scene)

We are attempting to determine the 3D pose of the arUco markers based on 2D image information, so we need to know some parameters of the imaging geometry.  We create a model of a central projection camera, using as many of the parameters as we know:
-  the focal length, in this case 4.25mm
-  the image size
- the principal point, where the optical axis passes through the image plane
- the pixel size, in this case 1.4μm

From the camera model we can derive the intrinsic parameter matrix

In [ ]:
camera = CentralCamera(f=4.25e-3, imagesize=(4032, 3024), pp=(2016, 1512), rho=1.4e-6)
camera.K

There are many different arUco marker families, here we are using `4x4_1000` which is a $4 \times 4$ grid of squares that can encode numbers from 0 to 9999.  We pass in the marker family, the camera intrinsic parameter matrix, and the side length of the marker.  This last parameter is important, because with perspective projection we cannot tell the difference between some large and distance or small and close.  Knowing the size help us estimate distance.

In [ ]:
markers = scene.fiducial("4x4_1000", K=camera.K, side=67e-3);

The return is a list of Marker objects, each has the id of the marker, and the coordinates of four corners which is enough to estimate the orientation of the planar marker in 3D , which is shown above.  We pick element 2 of the marker list  

In [ ]:
marker = markers[2]
marker.id

which is the marker with `id` of 5, the marker being held by the teddy bear.  The corners of this particular fiducial are

In [ ]:
marker.corners

and we can display them on the original image (zoomed in)

In [ ]:
scene.disp(block=None)
plot_point(marker.corners, 'bx')
plt.xlim(800, 1200)
plt.ylim(1200, 800)

The pose of the fiducial, with respect to the camera, as an SE(3) matrix is

In [ ]:
marker.pose

Finally, we can render a coordinate frame associated with the pose of each fiducial, into the original image.  The fiducial's z-axis is normal to its plane.

In [ ]:
for marker in markers:
    marker.draw(scene, length=0.10, thick=20)
scene.disp();